# Etiquetage et annotation

Collecte de données automatisée de notre dossier d'images (116 images) pour le système de recommandation selon 5 critères dans un fichier 'etiquetage.json'.
- Orientation de l'images 
- Résolution de l'image
- ISO de l'image
- La catégorie : chien ou chat
- Animal dans la nature ou non ? proportion de vert dans l'image

In [2]:
from pandas import json_normalize
import pandas as pd
import json
import math
import matplotlib.pyplot as plot
import numpy as np

## EXIF des images

Dans un premier temps on récupère 3 données Exif dans un fichier JSON : nom de l'image, orientation de l'images, résolution de l'image et ISO de l'image

In [51]:
import os
import json
from PIL import Image
from PIL.ExifTags import TAGS

# Définir le dossier contenant les images
image_folder = "images/"
file = "etiquetage.json"

# Liste pour stocker les métadonnées
metadata_list = []

# Fonction pour déterminer l'orientation de l'image
def get_orientation(image):
    width, height = image.size
    if width > height:
        return "Paysage"
    elif width < height:
        return "Portrait"
    else:
        return "Carre"

# Parcourir tous les fichiers du dossier
for filename in os.listdir(image_folder):
    file_path = os.path.join(image_folder, filename)
    
    # Vérifier si c'est un fichier image
    try:
        with Image.open(file_path) as img:            
            width, height = img.size  # Récupère le nombre de pixels

            if width*height < 600*600:
                quality = "basse"
            elif width*height > 1900*1900:
                quality = "haute"
            else:
                quality = "moyenne"

            image_info = {
                "nom_image": filename,
                "orientation": get_orientation(img),
                "qualite": quality,
            }

            metadata_list.append(image_info)

    except Exception as e:
        print(f"Erreur avec {filename} : {e}")

# Sauvegarde des métadonnées dans un fichier JSON
with open(file, "w", encoding="utf-8") as json_file:
    json.dump(metadata_list, json_file, indent=4, ensure_ascii=False)

print(f"Métadonnées enregistrées dans {file}")

Métadonnées enregistrées dans etiquetage.json


# Catégorie : chien ou chat

On utilise ....

In [3]:
!pip install tensorflow


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [52]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import decode_predictions
from PIL import ImageOps
import numpy as np
import os
import json

# Charger le modèle MobileNetV2 pré-entrainé
model = MobileNetV2(weights='imagenet')

# Liste des races de chiens et de chats pour élargir la classification
dog_races = ["terrier", "retriever", "beagle", "bulldog", "poodle", "chihuahua", "labrador", "rottweiler", "pug", "dalmatian", "dog", "puppy", "wolf", "coyote", "fox"]
cat_races = ["siamese", "persian", "maine coon", "bengal", "sphynx", "russian blue", "ragdoll", "british shorthair", "burmese", "devon rex", "cat", "kitten", "tiger", "lynx", "leopard"]

def prepare_image(image_path):
    # Charger l'image et la redimensionner directement à 224x224
    img = image.load_img(image_path, target_size=(224, 224))
    
    # Convertir l'image en tableau numpy pour prétraitement
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Ajouter une dimension pour le batch
    img_array = preprocess_input(img_array)  # Prétraitement spécifique pour MobileNetV2
    
    return img_array

def predict_image(image_path):
    # Préparer l'image
    preprocessed_image = prepare_image(image_path)
    
    # Faire la prédiction
    predictions = model.predict(preprocessed_image)
    
    # Décoder les prédictions (top 10)
    decoded_predictions = decode_predictions(predictions, top=10)[0]
    
    # Vérifier si l'une des 10 premières classes correspond à une race de chien ou de chat
    for _, label, _ in decoded_predictions:
        if any(dog_race in label.lower() for dog_race in dog_races):
            return 'Chien'
        elif any(cat_race in label.lower() for cat_race in cat_races):
            return 'Chat'
    
    # Si aucune prédiction ne correspond, retourner "Unknown"
    return 'Unknown'

In [53]:
# Charger les données existantes de etiquetage.json
if os.path.exists(file):
    with open(file, "r", encoding="utf-8") as f:
        metadata_list = json.load(f)
else:
    metadata_list = []

# Ajouter les prédictions à chaque image
for entry in metadata_list:
    # Chercher l'image correspondante dans le dossier
    image_name = entry.get("nom_image")
    if image_name:
        image_path = os.path.join(image_folder, image_name)
        
        if os.path.exists(image_path):  # Si le fichier image existe
            # Prédire si l'image est un chien ou un chat
            label = predict_image(image_path)
            
            # Ajouter la prédiction à l'élément de la liste
            entry['animal'] = label

# Sauvegarder les résultats dans etiquetage.json
with open(file, "w", encoding="utf-8") as f:
    json.dump(metadata_list, f, indent=4, ensure_ascii=False)

print(f"Résultats mis à jour dans {file}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━

## Couleurs

On analyse ...

In [6]:
!pip install scikit-learn

In [6]:
import os
from os import listdir
from PIL import Image
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import json

In [57]:
import os
import json
import numpy as np
from PIL import Image
import cv2

# Charger la liste des images dans l'ordre du système de fichiers
images = ["images/" + file for file in os.listdir("images/")]

# Charger les métadonnées existantes du fichier JSON
file = "etiquetage.json"
if os.path.exists(file):
    with open(file, "r", encoding="utf-8") as json_file:
        metadata = json.load(json_file)
else:
    metadata = []

# Seuils pour détecter le vert en HSV
def is_green_hsv(hue, saturation, value):
    min_hue = 35  # Plage ajustée pour inclure plus de verts
    max_hue = 90  # Ajuste la plage de teinte 
    min_saturation = 15  # Augmente la saturation minimale pour mieux capter les couleurs vives
    min_value = 40  # Augmente la valeur minimale pour exclure les pixels trop sombres
    
    return min_hue <= hue <= max_hue and saturation >= min_saturation and value >= min_value

# Détection de la nature pour chaque image et mise à jour du JSON
for img_path in images:
    img_name = os.path.basename(img_path)  
    imgfile = Image.open(img_path).convert("RGB")

    # Redimensionner l'image pour optimiser la vitesse
    img_resized = imgfile.resize((300, 300)) 
    img_array = np.array(img_resized)

    # Convertir l'image de RGB à HSV
    img_hsv = cv2.cvtColor(img_array, cv2.COLOR_RGB2HSV)

    # Nombre total de pixels
    total_pixels = img_hsv.shape[0] * img_hsv.shape[1]

    # Compter les pixels verts en fonction des seuils HSV
    green_pixels = 0
    for pixel in img_hsv.reshape((-1, 3)):  # Parcourir chaque pixel
        hue, saturation, value = pixel
        if is_green_hsv(hue, saturation, value):
            green_pixels += 1

    # Calcul du pourcentage de vert
    green_ratio = green_pixels / total_pixels

    # Déterminer si l’image est en nature ou pas
    nature_label = "en nature" if green_ratio >= 0.10 else "pas de nature"  

    # Mise à jour du JSON : Trouver l’entrée correspondante
    for entry in metadata:
        if entry.get("nom_image") == img_name:
            entry["nature"] = nature_label
            break  # Sortir dès qu'on a trouvé l'entrée correspondante

# Réécriture du fichier JSON avec les nouvelles valeurs
with open(file, "w", encoding="utf-8") as json_file:
    json.dump(metadata, json_file, ensure_ascii=False, indent=4)


In [58]:
def is_luminous(image_path, seuil):
    # Seuil de luminosité (0-255), au-dessus = lumineuse, en dessous = sombre
    # return True si lumineuse, False si sombre

    # Ouvrir l’image et la convertir en niveaux de gris
    img = Image.open(image_path).convert("L")

    # Convertir en tableau NumPy
    pixel_values = np.array(img)

    # Calculer la luminosité moyenne
    brightness = np.mean(pixel_values)

    # Comparer avec le seuil
    return brightness > seuil

lum = []
for img_path in images:
    if is_luminous(img_path, 128):
        lum.append("lumineux")
    else:
        lum.append("sombre")

In [59]:
# Le chemin vers ton fichier JSON
file = "etiquetage.json"

# Charger le fichier JSON existant
if os.path.exists(file):
    with open(file, "r", encoding="utf-8") as json_file:
        metadata = json.load(json_file)
else:
    metadata = []

# Ajouter une catégorie à chaque objet
i = 0
for entry in metadata:
    entry["luminosite"] = lum[i]
    i += 1

# Réécrire le fichier avec les données mises à jour
with open(file, "w", encoding="utf-8") as json_file:
    json.dump(metadata, json_file, ensure_ascii=False, indent=4)